In [2]:
import pandas as pd

import numpy as np

In [1]:
import requests, json, time

In [4]:
df_gsc  = pd.read_csv("../data/GSC_PLUS.csv") 

In [5]:
df_gsc.head()

,file,text,HPO_symptoms,HPO_codes,starts,ends
0,1003450,A syndrome of brachydactyly (absence of some m...,"['brachydactyly', 'absence of some middle or d...","['0001156', '0009881', '0001798', '0001792', '...","[14, 29, 74, 86, 105, 120, 166, 210, 232, 346,...","[27, 71, 103, 103, 118, 163, 208, 226, 253, 37..."
1,10051003,Townes-Brocks syndrome (TBS) is an autosomal d...,"['autosomal dominant disorder', 'autosomal dom...","['0000006', '0000006', '0000006', '0003828', '...","[35, 35, 45, 95, 95, 139, 148, 163, 177, 186, ...","[62, 53, 53, 114, 103, 161, 161, 175, 197, 197..."
2,10066029,Nevoid basal cell carcinoma syndrome (NBCCS) i...,"['basal cell carcinoma', 'autosomal dominant',...","['0002671', '0000006', '0000006', '0000006', '...","[7, 89, 89, 99, 99, 143, 143, 208, 238, 250, 2...","[27, 107, 113, 113, 107, 164, 151, 229, 261, 2..."
3,10196695,Angelman syndrome (AS) is a neurodevelopmental...,"['neurodevelopmental disorder', 'contiguous ge...","['0000707', '0001466']","[28, 839]","[55, 863]"
4,10417280,Prader-Willi syndrome (PWS) and Angelman syndr...,"['neurobehavioral disorders', 'de novo']","['0000708', '0003745']","[68, 223]","[93, 230]"


In [9]:
df_gsc.shape

(228, 6)

In [89]:
payload = {"documents":[]}
response = {"documents"}
for i, r in df_gsc.iterrows():
    doc = {"language":"en", "id":i, "text": r["text"]}
    payload["documents"].append(doc)
    if i > 3: break

In [81]:
payload

{'documents': [{'language': 'en',
   'id': 0,
   'text': 'A syndrome of brachydactyly (absence of some middle or distal phalanges), aplastic or hypoplastic nails, symphalangism (ankylois of proximal interphalangeal joints), synostosis of some carpal and tarsal bones, craniosynostosis, and dysplastic hip joints is reported in five members of an Italian family. It may represent a previously undescribed autosomal dominant trait.'},
  {'language': 'en',
   'id': 1,
   'text': 'Townes-Brocks syndrome (TBS) is an autosomal dominant disorder with multiple malformations and variable expression. Major findings include external ear anomalies, hearing loss, preaxial polydactyly and triphalangeal thumbs, imperforate anus, and renal malformations. Most patients with Townes-Brocks syndrome have normal intelligence, although mental retardation has been noted in a few.'},
  {'language': 'en',
   'id': 2,
   'text': 'Nevoid basal cell carcinoma syndrome (NBCCS) is a hereditary condition transmitted as 

In [95]:
#TEXTANALYTICS_URL = "http://hacksymptoms-appservice.azurewebsites.net/text/analytics/v3.0-preview.1/domains/health"
TEXTANALYTICS_URL="https://ta4h-app-service.azurewebsites.net/text/analytics/v3.0-preview.1/domains/health"
HEADERS = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
}


In [96]:
BATCH_SIZE = 3
responses = {"documents":[]}
n_in_batch = 1
for i, r in df_gsc.iterrows():
    payload = {"documents":[]}
    doc = {"language":"en", "id":i, "text": r["text"]}
    payload["documents"].append(doc)
    
    n_in_batch += 1
    if (n_in_batch == BATCH_SIZE):
        response = requests.request("POST", TEXTANALYTICS_URL, headers=HEADERS, json = payload)
        responses["documents"].append(response.json()["documents"])
        n_in_batch = 1
        time.sleep(3)

In [ ]:
responses

In [ ]:
import pickle


In [99]:
with open('text_analytics_responses.pickle', 'wb') as f:
    pickle.dump(responses, f)

In [3]:
df_gsc = pd.read_pickle("text_analytics_responses.pickle")

In [6]:
df_gsc["documents"][0:10]

[[{'id': '1',
   'entities': [{'id': '0',
     'offset': 0,
     'length': 22,
     'text': 'Townes-Brocks syndrome',
     'type': 'DIAGNOSIS',
     'score': 1.0,
     'isNegated': False,
     'links': [{'dataSource': 'UMLS', 'id': 'C0265246'},
      {'dataSource': 'CHV', 'id': '0000025864'},
      {'dataSource': 'JABL', 'id': '631'},
      {'dataSource': 'MSH', 'id': 'C536974'},
      {'dataSource': 'NCI', 'id': 'C99085'},
      {'dataSource': 'NCI_NICHD', 'id': 'C99085'},
      {'dataSource': 'OMIM', 'id': '107480'},
      {'dataSource': 'SNMI', 'id': 'D4-00802'},
      {'dataSource': 'SNOMEDCT_US', 'id': '24750000'}]},
    {'id': '1',
     'offset': 24,
     'length': 3,
     'text': 'TBS',
     'type': 'DIAGNOSIS',
     'score': 0.99,
     'isNegated': False,
     'links': [{'dataSource': 'UMLS', 'id': 'C0265246'},
      {'dataSource': 'CHV', 'id': '0000025864'},
      {'dataSource': 'JABL', 'id': '631'},
      {'dataSource': 'MSH', 'id': 'C536974'},
      {'dataSource': 'NCI', 'id